In [ ]:
### PROCEDURES ###
#   Configerations
#        |
#        |
# Galaxy distribution
#        |
#        |
#    CosmoSIS
#        |
#        |
#    

In [ ]:
print('Started. Loading libraries ... ')
import os,sys,inspect
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as intg
from scipy import interpolate
import pandas as pd
import fitsio as fio

# #import modules from parent directory
# currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# parentdir = os.path.dirname(currentdir)
# sys.path.insert(0,parentdir) 
from interpolation_fncs import *
from io_fncs import *
from gal_distr import *
from basic_cosmo_lensing_fncs import *
# from limber_integral import *

########################################
#io
########################################

outputfolder       = 'chi_based_SC'
outputparentfolder = '/hpc/group/cosmology/alanzhou/IA-Project/IA-Self-Calib-Data/'
outputpath         = os.path.join(outputparentfolder, outputfolder)
create_dir(outputpath)
print('output path set to %s'%(outputpath))

########################################
#constants
########################################

light_speed = 299792.458 #km/s
H0 = 67.26 #H0 (km/s/Mpc)
d_horiz = light_speed/H0 #c/H0, c in km/s
Omega0 = 0.3141 #present day matter density in unit of critical density
OmegaLambda = 1-0.3141
Omegak = 1-Omega0-OmegaLambda
deltaz = 0.2 #redshift bin width
#in integration 1 redshift is broken into z_res intervals
#for z_max = 4, zres = 400 ~20sec/run
z_bin_num = 20
z_max = z_bin_num*deltaz+2 #+2 redshift reserved for photometric error
z_res = 100 
gal_distr_model = 'z2exp_gausPDF'

save_config_file(os.path.join(outputpath,'values.txt'),
                 ['Identifier',outputfolder,
                  'H0, [km/s/Mpc]',H0,
                  'horizon distance = c/H0, [Mpc]',d_horiz,
                  'speed of light [km/s]',light_speed,
                  'Omega0',Omega0,
                  'OmegaLambda',OmegaLambda,
                  'Omegak',Omegak,
                  'deltaz',deltaz,
                  'number of galaxy redshift bins',z_bin_num,
                  'max z used in calculations',z_max,
                  'z resolution [intervals/ unit z]',z_res,
                  'z resolution spacing [z]',1/z_res,
                  'galaxy distribution model',gal_distr_model,
                  'output path', outputpath])

print(' - Setup complete. z resolution is %1.5f' %(1/z_res))

########################################
#Pre-CosmoSIS Calculations
########################################

#get the galaxy distribution
print(' - Getting the galaxy distribution, assuming '+gal_distr_model)
[photo_z_file,true_z_file] = get_gal_distr(z_bin_num,deltaz,z_res,z_max,gal_distr_model,outputpath);
z_list = photo_z_file['Z_MID']


#get cmv distance and lensing kernel (unit = Mpc)
#get integrated lensing kernel W_i(z_L) for each redsfhit bin (Zhang 2010 A2)
#chi tested, exactly what cosmoSIS produces
print(' - Computing cmv dists [Mpc], W_L, and integrated lensing kernels W_i')
[chi_list,W_L_list] = get_basic_cosmo_lensing(d_horiz,Omega0,OmegaLambda,Omegak,z_list,outputpath,USECOSMOSIS=True,data_path=os.path.join(outputpath, 'cosmoSIS_output/distances')); #in Mpc unit
W_i = get_integrated_lensing_kernel(W_L_list,true_z_file,z_list,z_bin_num,outputpath)

########################################
#Load CosmoSIS Output
########################################

#run the matter_pow.ini with CosmoSIS from terminal
#the cosmoSIS has to be set up with the same cosmology
#the max z, max ell in CAMB must be at least span the z_list's range used here
print(' - Assuming CosmoSIS has being ran. Loading CosmoSIS output.')

#load hubble parameter
print('   - Loading h(z) from CosmoSIS, computing H(Z) [km/s/Mpc] and interpolating.')
#CosmoSIS only explicitly gives the little h, in unit Mpc^-1
#H = c 100 h km s^-1 Mpc^-1
distance_folder = os.path.join(outputpath,'cosmoSIS_output/distances')
with open(os.path.join(distance_folder,'z.txt'),'rb') as f:
    temp_z_h = np.loadtxt(f)
with open(os.path.join(distance_folder,'h.txt'),'rb') as f:
    temp_little_h = np.loadtxt(f)
hubble_func = interpolate.interp1d(temp_z_h,temp_little_h,kind='linear') 
hubble_list = hubble_func(z_list)*light_speed #this is big H(z) in unit of km/s/Mpc

#load the C_GG in angular space from CosmoSIS (shear-shear option)
print('   - Loading C_GG(ell) from CosmoSIS')
shear_cl_folder = os.path.join(outputpath,'cosmoSIS_output/shear_cl')
with open(os.path.join(shear_cl_folder,'ell.txt'),'rb') as f:
    ell_list = np.loadtxt(f)
C_GG_ii = np.empty((z_bin_num,len(ell_list)))
plt.subplots(figsize=(6,6))
for i in range(z_bin_num):
    with open(os.path.join(shear_cl_folder,'bin_%i_%i.txt'%(i+1,i+1)),'rb') as f:
        C_GG_ii[i] = np.loadtxt(f)
    plt.plot(ell_list,C_GG_ii[i],label='C_GG_ii, i=%i'%(i+1))
plt.xlabel('ell')
plt.ylabel('C_GG_ii')
plt.xscale('log')
plt.yscale('log')
plt.legend(bbox_to_anchor=(1, 0),loc='lower left') #bbox(x,y)
plt.tight_layout()
plt.savefig(os.path.join(outputpath,'combined_C_GG_ii.png'))
#plt.show()
plt.close()

#load the C_Gg in angular space from CosmoSIS (position-shear option)
print('   - Loading C_Gg(ell) from CosmoSIS')
galaxy_shear_cl_folder = os.path.join(outputpath,'cosmoSIS_output/galaxy_shear_cl')
with open(os.path.join(galaxy_shear_cl_folder,'ell.txt'),'rb') as f:
    ell_list = np.loadtxt(f)
C_Gg_ii = np.empty((z_bin_num,len(ell_list)))
plt.subplots(figsize=(6,6))
for i in range(z_bin_num):
    with open(os.path.join(galaxy_shear_cl_folder,'bin_%i_%i.txt'%(i+1,i+1)),'rb') as f:
        C_Gg_ii[i] = np.loadtxt(f)
    plt.plot(ell_list,C_Gg_ii[i],label='C_Gg_ii, i=%i'%(i+1))
plt.xlabel('ell')
plt.ylabel('C_Gg_ii')
plt.xscale('log')
plt.yscale('log')
plt.legend(bbox_to_anchor=(1, 0),loc='lower left') #bbox(x,y)
plt.tight_layout()
plt.savefig(os.path.join(outputpath,'combined_C_Gg_ii.png'))
#plt.show()
plt.close()

#load the C_gg in angular space from CosmoSIS (position-position option)
print('   - Loading C_gg(ell) from CosmoSIS')
galaxy_cl_folder = os.path.join(outputpath,'cosmoSIS_output/galaxy_cl')
with open(os.path.join(galaxy_cl_folder,'ell.txt'),'rb') as f:
    ell_list = np.loadtxt(f)
C_gg_ii = np.empty((z_bin_num,len(ell_list)))
plt.subplots(figsize=(6,6))
for i in range(z_bin_num):
    with open(os.path.join(galaxy_cl_folder,'bin_%i_%i.txt'%(i+1,i+1)),'rb') as f:
        C_gg_ii[i] = np.loadtxt(f)
    plt.plot(ell_list,C_gg_ii[i],label='C_gg_ii, i=%i'%(i+1))
plt.xlabel('ell')
plt.ylabel('C_gg_ii')
plt.xscale('log')
plt.yscale('log')
plt.legend(bbox_to_anchor=(1, 0),loc='lower left') #bbox(x,y)
plt.tight_layout()
plt.savefig(os.path.join(outputpath,'combined_C_gg_ii.png'))
#plt.show()
plt.close()

#get the power spectrum P(k,z) from CosmoSIS. (need non-linear spectrum)
#CosmoSIS unit: k is in (Mpc/h), P is in (h/Mpc)^3
#interpolating the P(k,z) to fit a z_list of Z_MID values
#matter_pow_p_k[z_index,kindex]
print('   - Loading P(k,z) from CosmoSIS.')
matter_power_folder = os.path.join(outputpath,'cosmoSIS_output/matter_power_nl')
with open(os.path.join(matter_power_folder, 'p_k.txt'),'rb') as f:
    matter_pow_p_k = np.loadtxt(f,delimiter = ' ') 
with open(os.path.join(matter_power_folder, 'k_h.txt'),'rb') as f:
    matter_pow_k_h = np.loadtxt(f,delimiter = ' ')
with open(os.path.join(matter_power_folder, 'z.txt'),'rb') as f:
    matter_pow_z = np.loadtxt(f,delimiter = ' ')
print(' - Interpolating P(k,z) for further calculations...')
matter_pow_p_k_interpolated = interpolate_2d(matter_pow_p_k,matter_pow_z,matter_pow_k_h,z_list,matter_pow_k_h)

In [ ]:
#